In [1]:
import numpy as np
import spacy
import gensim
import os
import re
from gensim.utils import simple_preprocess, lemmatize
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd
nlp = spacy.load("en_core_web_md")

import ebooklib
from ebooklib import epub

C:\Users\tally\.virtualenvs\Repositories-J01Zxu1S\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from bs4 import BeautifulSoup
from os import mkdir
import requests
#Dicitionary used to access full book text in HTML format
urls = {'foucault_madness_and_civ':'https://archive.org/stream/Michel_Foucault_Madness_And_Civilization/Michel%20Foucault%2C%20Richard%20Howard%20%28transl.%29%20-%20Madness%20and%20Civilization_%20A%20History%20of%20Insanity%20in%20the%20Age%20of%20Reason%20%282013%2C%20Vintage%29_djvu.txt',
        'foucault_history_of_sexuality':'https://archive.org/stream/TheHistoryOfSexualityVol13/The-History-Of-Sexuality-Vol-1-3_djvu.txt',
        'chomsky_american_power': 'https://archive.org/stream/AmericanPowerAndTheNewMandarins_201805/American%20Power%20And%20The%20New%20Mandarins_djvu.txt',
        'chomsky_manufacturing_consent': 'https://archive.org/stream/revhosatx14/%5BEdward_S._Herman%2C_Noam_Chomsky%5D_Manufacturing_Con%28b-ok.org%29_djvu.txt'}

In [19]:
test_url = 'https://archive.org/download/TheHistoryOfSexualityVol13/The-History-Of-Sexuality-Vol-1-3.epub'
book = epub.read_epub('TheHistoryOfSexualityVol13_The-History-Of-Sexuality-Vol-1-3.epub')
for doc in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
    print(doc)

<EpubHtml:cover:cover.xhtml>
<EpubHtml:chapter_0:chap_0001.xhtml>
<EpubHtml:chapter_1:chap_0002.xhtml>
<EpubHtml:chapter_2:chap_0003.xhtml>
<EpubHtml:chapter_3:chap_0004.xhtml>
<EpubHtml:chapter_4:chap_0005.xhtml>
<EpubHtml:chapter_5:chap_0006.xhtml>
<EpubHtml:chapter_6:chap_0007.xhtml>
<EpubHtml:chapter_7:chap_0008.xhtml>
<EpubHtml:chapter_8:chap_0009.xhtml>
<EpubHtml:chapter_9:chap_0010.xhtml>
<EpubHtml:chapter_10:chap_0011.xhtml>
<EpubHtml:chapter_11:chap_0012.xhtml>
<EpubHtml:chapter_12:chap_0013.xhtml>
<EpubHtml:chapter_13:chap_0014.xhtml>
<EpubHtml:chapter_14:chap_0015.xhtml>
<EpubHtml:chapter_15:chap_0016.xhtml>
<EpubHtml:chapter_16:chap_0017.xhtml>
<EpubHtml:chapter_17:chap_0018.xhtml>
<EpubHtml:chapter_18:chap_0019.xhtml>
<EpubHtml:chapter_19:chap_0020.xhtml>
<EpubHtml:chapter_20:chap_0021.xhtml>
<EpubHtml:chapter_21:chap_0022.xhtml>
<EpubHtml:chapter_22:chap_0023.xhtml>
<EpubHtml:chapter_23:chap_0024.xhtml>
<EpubHtml:chapter_24:chap_0025.xhtml>
<EpubHtml:chapter_25:chap_0026.

In [25]:
def tokenize(doc):
    '''Tokenizer with lemmatizer'''
    return [token.lemma_ for token in nlp(doc) if (token.is_stop == False) & (token.is_punct == False)]

def gather_data(path_to_data):
    data = []
    #For file at the given path
    for file in os.listdir(path_to_data):
        #If the directory is not a folder
        if os.path.isdir(file) == False:
            #If the file type is .txt
            if file[-3:] == 'txt':
                #Open each text file at the path provided
                with open(os.path.join(path_to_data, file), encoding='utf-8') as t:
                    #Read and strip new line signal
                    text = t.read().replace('\n', ' ')
                    data.append(str(text))
    return data

In [26]:
tokenize('Hello its me I find you here on the very next day you gave it away')      

['hello', 'find', 'day', 'give', 'away']

In [27]:
df = pd.DataFrame()

In [28]:
pages_folders = [folder for folder in os.listdir('./') if (os.path.isdir(folder) == True) & (folder != '.git') & (folder != '.ipynb_checkpoints')]

In [29]:
df.append(gather_data(pages_folders[0]))

,0
0,
1,AMERICAN POWER AND THE NEW MANDARINS chan...
2,Objectivity and Liberal Scholarship The d...
3,AMERICAN POWER AND THE NEW MANDARINS slee...
4,Objectivity and Liberal Scholarship simpl...
5,AMERICAN POWER AND THE NEW MANDARINS ge...
6,Objectivity and Liberal Scholarship those...
7,* - AMERICAN POWER AND THE NEW MANDARINS I...
8,Objectivity and Liberal Scholarship 1937 ...
9,AMERICAN POWER AND THE NEW MANDARINS and ...


In [42]:
for book in pages_folders:
    df.append(gathe)

In [47]:
data_dict['chomsky_american_power']

['             NOAM CHOMSKY    VINTAGE BOOKS  A DIVISION OF RANDOM HOUSE  NEW YORK      ',
 ' ',
 '    ★ ——    AMERICAN POWER AND THE NEW MANDARINS    Whereas Jackson attributes the ebbing of the revolutionary  tide to the discovery of the unsuspected complexity of modern  society, Orwell’s firsthand observations, like those of Borkenau,  suggest a far simpler explanation. What calls for explanation is  not the disaffection of the workers of Barcelona but the curious  constructions of the historian.   Let me repeat, at this point, Jackson’s comments regarding  Juan Comorera: Comorera “immediately took steps to end  barter and requisitioning, and became a defender of the peas¬  ants against the revolution”; he “ended requisitions, restored  money payments, and protected the Catalan peasants against  further collectivization.” These comments imply that the peas¬  antry of Catalonia was, as a body, opposed to the revolution  and that Comorera put a stop to the collectivization that they  

In [45]:
df

,0
0,NOAM CHOMSKY VINTAGE BOOKS A ...
1,
2,★ —— AMERICAN POWER AND THE NEW MANDARI...
3,simply been compounded in more recent histo...
4,Objectivity and Liberal Scholarship those...
5,Objectivity and Liberal Schola...
6,★= AMERICAN POWER AND THE NEW MANDARINS a...
7,Objectivity and Liberal Scholar...
8,
9,AMERICAN POWER AND THE NEW MANDARINS the ...
